In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
D1_LINEAR_GAUSIAN_250px = "D://250px_LI_GB_D1/"
D2_LINEAR_GAUSIAN_250px = "D://250px_LI_GB_D2/"
ds_type = D1_LINEAR_GAUSIAN_250px
dataset_path = signature_types = os.listdir(ds_type)
signs = []
for item in signature_types:
  all_signs = os.listdir(ds_type+"/"+item)
  for sign in all_signs:
    if sign.endswith(".db"):
      pass
    else: 
      signs.append((item, str(ds_type+"/"+item) + "/" + sign))
signs_df = pd.DataFrame(data = signs, columns = ["sign type", "image"])
sign_count = signs_df["sign type"].value_counts()
print("Signatures in each category:")
print(sign_count)

In [ ]:
images = []
labels = []
for i in signature_types:
  data_path = ds_type + str(i)
  filenames = [i for i in os.listdir(data_path)]
  for f in tqdm(filenames, desc=f"Processing {i}"):
    if f.endswith(".db"):
      pass
    else:
      img = cv2.imread(data_path + "/" + f)
      images.append(img)
      labels.append(i)

In [ ]:
images = np.array(images)
images = images.astype('float32') / 255.0

In [ ]:
y = signs_df['sign type'].values
y = y.reshape(-1,1)
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0])], remainder='passthrough')
Y=np.array(columnTransformer.fit_transform(y),dtype='float32')
print(Y.shape)

In [ ]:
images, Y = shuffle(images,Y,random_state=42)
train_x,test_x,train_y,test_y = train_test_split(images, Y, test_size=0.2, random_state=42)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
train_x = train_x / 255.0
test_x = test_x / 255.0

In [ ]:
from tensorflow.keras.applications import VGG16
# from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Model, Sequential

In [ ]:
image_size = [250,250]
model = VGG16(input_shape=image_size+[3], include_top=False, weights='imagenet')

In [ ]:
for layers in model.layers:
  layers.trainable = False

In [ ]:
final_arc = Model(inputs=model.input,outputs=Dense(2,activation='softmax')(Flatten()(model.output)))

In [ ]:
final_arc.compile(optimizer='adam',loss='CategoricalCrossentropy',metrics=['accuracy'])

In [ ]:
history = final_arc.fit(train_x, train_y, validation_data=(test_x,test_y), epochs = 20)

In [ ]:
training_acc = history.history['accuracy']
test_acc = history.history['val_accuracy']
epoch_count = range(1, len(training_acc) + 1)
plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
testing = final_arc.evaluate(test_x, test_y)
predictions = final_arc.predict(test_x)
pred_y = np.argmax(predictions, axis = 1)
test_y = np.argmax(test_y, axis=1)
print(classification_report(test_y,pred_y))

In [ ]:
# cm = confusion_matrix(test_y, pred_y)
# print(cm)
cm = tf.math.confusion_matrix(test_y, pred_y)
print(cm)